<a href="https://colab.research.google.com/github/jesusGS01/TGINE/blob/main/3_2_scrapyJSON_LD_Elmundo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 3 - Scrapy - Ejemplo Crawling JSON-LD (ElMundo)

## Apartado 1.1 Crawler de El Mundo extrayendo JSON-LD
En el ejemplo siguiente definimos un crawler en Scrapy para extraer noticias de los metadatos de las páginas web.
El Mundo y otros periódicos publican metadatos en formato JSON-LD (https://json-ld.org/) que permite obtener información estructurada de las webs. En este caso, tendremos que obtener esos objetos JSON-LD y extraer su información en el formato de noticia (https://schema.org/NewsArticle) publicado en Schema.org

** Existen librerías de Python para trabajar directamente con JSON-LD

In [ ]:
!pip3 install -U scrapy

In [1]:
import scrapy
import sys
import json
import locale
import time
import random
from bs4 import BeautifulSoup

class ElMundoSpider (scrapy.Spider):

    #Es obligatorio poner un nombre
    name = 'elmundo'

    #Ponemos que el dominio que está permitido es el de la página y no vamos a irnos fuera de la misma
    allowed_domains = ['www.elmundo.es']

    start_urls = ['https://www.elmundo.es/']

    # para evitar que el sitio te bloquee por usar scrapy es interesante cambiar el USER_AGENT
    # El user agent por defecto de Scrapy cuando hace una petición es
    # Scrapy/VERSION (+https://scrapy.org)
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    def parse (self, response):
      """
      @inherit

      @param self
      @param response
      """

      # Obtenemos la URL de la web que estamos procesando
      url = str(response.request.url).strip()

      # Obtenemos usando CSS la parte donde está definido el JSON-LD en la web
      for item in response.css('script[type=\"application/ld+json\"]'):
        data = json.loads(item.css('::text').get())
        # Procesamos aquí los datos
        # RELLENAR
        if 'headline' in data:
          title = data['headline']
          content = data['description']
          date = data['datePublished']
          print ("-------------------------")
          print (url)
          print (title)
          print (content)
          print (date)
          print ("-------------------------")

          data = {
              'url' : url,
              'title': title,
              'content': content,
              'date': date
          }

          #Creamos para cada item un fichero json y para ello obtenemos un número aleatorio.
          filename = str(random.random()).replace(".","") + ".json"

          # Guardamos el documento si tiene contenido y título
          if content and title:
              with open ('elmundo/' + filename, 'w') as f:
                  json.dump (data, f, indent = 4)

      # Por último, miramos distintos enlaces que existan en la web (en este caso todos los de la etiqueta <a>)
      url_in_current_document = response.css ('a')
      for next_page in url_in_current_document:
          # Por cada enlace ejecutamos el mismo parse

          #Intentamos filtrar que solamente coja deporte
          url_str= str(next_page.css('::attr(href)').get())
          if url_str.find("elmundo.es/deportes") != -1:
            yield response.follow (next_page, self.parse)


## Apartado 1.2
Para poder lanzar el Spider necesitamos que ejecutar el siguiente código donde se configuará y lanzará el proceso.
Hay que hacer notar que solamente se puede lanzar un proceso por cada sesión en Jupyter notebook es por eso por lo que se recomienda exportar el código en un script de Python .py para poder ejecutarlo desde la línea de comandos.

In [2]:
import os
import scrapy
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if (not os.path.exists('elmundo')):
    os.mkdir('elmundo')

# Creamos el proceso con el RSSSpider
process.crawl(ElMundoSpider)
# Ejecutamos el Crawler
process.start()

INFO:scrapy.utils.log:Scrapy 2.11.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.5, Platform Linux-5.15.120+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)
DEBUG:scrap

-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/futbol.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/futbol/primera-division.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/futbol/segunda-division.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/futbol/champions-league.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/futbol/europa-league.html
-------------------------
-------------------------

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/futbol/europa-league.html> (referer: https://www.elmundo.es/)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/futbol/primera-division.html> (referer: https://www.elmundo.es/)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/futbol.html> (referer: https://www.elmundo.es/)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/futbol/premier-league.html> (referer: https://www.elmundo.es/)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/futbol/segunda-division.html> (referer: https://www.elmundo.es/)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/futbol/liga-portuguesa.html> (referer: https://www.elmundo.es/)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/futbol/liga-argentina.html> (referer: https://www.elmundo.es/)
DEBUG:scrapy.core.engine:Crawled (200)

-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/futbol/2023/10/26/653a467a21efa043148b4590.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/futbol.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/futbol/primera-division.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/futbol/segunda-division.html
-------------------------
-------------------------
L

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/baloncesto/nba.html> (referer: https://www.elmundo.es/)


...

-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/ciclismo.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/ciclismo/giro-de-italia.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/ciclismo/tour-de-francia.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/ciclismo/vuelta-a-espana.html
-------------------------
-------------------------
La proxima url que voy a estudiar esmailto:?subject=Sorpresas%20en%20C%C3%B3rdoba%20y%20%27n%C3%BAmero%20uno%27%20asegurado%20para%20las%20Martas&body=https://www.elmundo.es/deportes/padel/2019/11/15/5dcef033fc6c839e0c8b4636.html?emk=MAILSHARE
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes.html
----------------------

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.elmundo.es/deportes/ciclismo.html?intcmp=MENUDES22301&s_kw=ciclismo> (referer: https://www.elmundo.es/deportes/padel/2019/11/03/5dbee69afdddffbc2b8b4575.html)


Se han truncado las últimas 5000 líneas del flujo de salida.
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/ciclismo/vuelta-a-espana/clasificacion/general.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/ciclismo/vuelta-a-espana/clasificacion/etapa-1.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/ciclismo/vuelta-a-espana/palmares.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/ciclismo/vuelta-a-espana/2023/09/17/6505c8f721efa0ee478b4583.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/ciclismo/vuelta-a-espana/2023/09/17/6505c8f721efa0ee478b4583.html#ancla_comentarios
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://ww

INFO:scrapy.statscollectors:Dumping Scrapy stats:
{'downloader/request_bytes': 26252,
 'downloader/request_count': 68,
 'downloader/request_method_count/GET': 68,
 'downloader/response_bytes': 3911033,
 'downloader/response_count': 68,
 'downloader/response_status_count/200': 68,
 'dupefilter/filtered': 8063,
 'elapsed_time_seconds': 23.316556,
 'finish_reason': 'shutdown',
 'finish_time': datetime.datetime(2023, 10, 26, 17, 5, 27, 59613, tzinfo=datetime.timezone.utc),
 'httpcompression/response_bytes': 21097291,
 'httpcompression/response_count': 68,
 'log_count/DEBUG': 70,
 'log_count/INFO': 11,
 'memusage/max': 172347392,
 'memusage/startup': 172347392,
 'offsite/filtered': 25,
 'request_depth_max': 4,
 'response_received_count': 68,
 'scheduler/dequeued': 68,
 'scheduler/dequeued/memory': 68,
 'scheduler/enqueued': 1645,
 'scheduler/enqueued/memory': 1645,
 'start_time': datetime.datetime(2023, 10, 26, 17, 5, 3, 743057, tzinfo=datetime.timezone.utc)}
INFO:scrapy.core.engine:Spider 

-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/futbol/futbol-femenino/2023/08/25/64e8af71fdddff0f1c8b458b.html#ancla_comentarios
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/futbol/futbol-femenino/2023/08/25/64e8af71fdddff0f1c8b458b.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/futbol/2023/08/23/64e60727e9cf4a546a8b45da.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/futbol/2023/08/23/64e60727e9cf4a546a8b45da.html#ancla_comentarios
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deportes/futbol/2023/10/25/653918f2e85ecead788b45de.html
-------------------------
-------------------------
La proxima url que voy a estudiar eshttps://www.elmundo.es/deporte